# Notebook Server Setup

In order to configure the notebook server, first the latest version of the AWS CLI v2 needs to be installed.

First check to see if the v2 is already installed. Don't worry if it is not installed, the notebook will install it if it not.

## Validate the AWS CLI v2 is installed.

You should see the following result, or something similar, when the next cell is executed:
```
aws-cli/2.x.x Python/3.x.x Linux/4.14.287-148.504.amzn1.x86_64 exe/x86_64.amzn.2018 prompt/off
```
Validate that the return value starts with aws-cli-/2.x.x

In [1]:
!aws --version

aws-cli/2.8.6 Python/3.9.11 Linux/4.14.294-150.533.amzn1.x86_64 exe/x86_64.amzn.2018 prompt/off


In [2]:
result = !aws --version
print(result[0])

if not "aws-cli/2" in result[0]:
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -qq awscliv2.zip
    !sudo ./aws/install --update
    !rm -rf aws
    !rm awscliv2.zip

aws-cli/2.8.6 Python/3.9.11 Linux/4.14.294-150.533.amzn1.x86_64 exe/x86_64.amzn.2018 prompt/off


# Test SSO Login

The instance has all the permissions required, now run the login command for the `Jupyter-IR-ViewOnly` role and the loggin account.

In [3]:
from jupyterirtools import sso

sso.login("Jupyter-IR-ViewOnly")

If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=TTBD-KZDR to activate the session


<IPython.core.display.Javascript object>

In [4]:
#session = boto3.Session(profile_name='ManagedPermissionSet-383086473915')
import boto3
boto3.setup_default_session()
sts = boto3.client('sts')
sts.get_caller_identity()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


{'UserId': 'AROAUCEELB36ZS4TV3HLF:mikeg',
 'Account': '279450423037',
 'Arn': 'arn:aws:sts::279450423037:assumed-role/AWSReservedSSO_Jupyter-IR-ViewOnly_6c1e764e56b1f7e0/mikeg',
 'ResponseMetadata': {'RequestId': '3473fe11-a711-4911-81ad-e607b1544b0c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3473fe11-a711-4911-81ad-e607b1544b0c',
   'content-type': 'text/xml',
   'content-length': '468',
   'date': 'Fri, 28 Oct 2022 20:47:49 GMT'},
  'RetryAttempts': 0}}

In [5]:
! aws sts get-caller-identity


{
    "UserId": "AROAUCEELB36ZS4TV3HLF:mikeg",
    "Account": "279450423037",
    "Arn": "arn:aws:sts::279450423037:assumed-role/AWSReservedSSO_Jupyter-IR-ViewOnly_6c1e764e56b1f7e0/mikeg"
}


## Validate permissions to IAM Identity Center

**NOTE-MOVE TO ORGANIZATION VALIDATION NOTEBOOK** 

The default profile needs access to the IAM identity center to create the profiles on the notebook server for the AWS CLI and SDK. The profiles allow the notebook to configure which profile will be used, allowing the notebooks to be used across accounts.

In the event that a profile is not configured on the notebook server, the notebook server will use the default profile to query  IAM Identity Center all the accounts and permission sets, and construct the profiles. This eliminates the need to call `aws configure sso` for every permutation of account and permission set.

If you get the error **ERROR: No Accounts Associated to upyter-IR-ViewOnly permission set.**, that means there are no AWS accounts associated with the **Jupyter-IR-ViewOnly** permission set. Go into the IAM Identity Center and associate the AWS account to the **Jupyter-IR-ViewOnly** permission set and a user or group.

In [8]:
from jupyterirtools import sso

admin_session = sso.get_management_session('Jupyter-IR-AdministratorAccess')

sso_admin_client = admin_session.client('sso-admin')
instances = sso_admin_client.list_instances()

sso_instance_arn = instances['Instances'][0]['InstanceArn']


permission_sets = sso_admin_client.list_permission_sets(InstanceArn=sso_instance_arn)

test_account = ""

for permission_set in permission_sets["PermissionSets"]:
    permission_set_detail = sso_admin_client.describe_permission_set(
            InstanceArn=sso_instance_arn,
            PermissionSetArn=permission_set
        )
    accounts = sso_admin_client.list_accounts_for_provisioned_permission_set(
            InstanceArn=sso_instance_arn,
            PermissionSetArn=permission_set
        )
    if permission_set_detail["PermissionSet"]["Name"] == "Jupyter-IR-ViewOnly":
        if len(accounts["AccountIds"]) == 0:
            print(f'ERROR: No Accounts Associated to {permission_set_detail["PermissionSet"]["Name"]} permission set')
        test_account = accounts["AccountIds"][0]
    print(f'Permission Set: {permission_set_detail["PermissionSet"]["Name"]} - Accounts: {accounts["AccountIds"]}')
    

Permission Set: Jupyter-IR-ViewOnly - Accounts: ['017990242238', '279450423037', '329925206794', '569230450028', '613375713311', '811741334843', '825219536996', '963668006045']
Permission Set: Jupyter-IR-AdministratorAccess - Accounts: ['017990242238', '279450423037', '329925206794', '569230450028', '613375713311', '811741334843', '825219536996', '963668006045']
Permission Set: AWSPowerUserAccess - Accounts: ['017990242238', '279450423037', '329925206794', '569230450028', '613375713311', '811741334843', '963668006045']
Permission Set: AWSReadOnlyAccess - Accounts: ['017990242238', '279450423037', '329925206794', '569230450028', '613375713311', '811741334843', '963668006045']
Permission Set: AWSServiceCatalogAdminFullAccess - Accounts: ['613375713311']
Permission Set: AWSAdministratorAccess - Accounts: ['017990242238', '279450423037', '329925206794', '569230450028', '613375713311', '811741334843', '825219536996', '963668006045']
Permission Set: AWSOrganizationsFullAccess - Accounts: ['0

In [9]:
!aws sso logout

In [10]:
boto3.setup_default_session()
sts = boto3.client('sts')
try:
    sts.get_caller_identity()
    print("The logout FAILED")
except:
    print("The logout was successful")

The logout was successful
